## <span style="color: purple">Create a SQL query to help Jane retrieving the monthly balance of all accounts</span>
our colleague Jane Hopper, the analyst in charge of analysing customer behaviour, who directly consumes data from the Data Warehouse Environment, needs to get all the account's monthly balances between Jan/2020 and Dec/2020. She wasn't able to do it alone, and asked for your help. Add to your resolution the SQL query used to retrieve the data needed 

I decided to use duckdb as the dialect. Duckdb is a powerful in-process, vectorized database, with great integration with Python. To promote best practices and easy comprehension, I created a module called utils.__main__ in the utils folder. It contains one function to parse the CSV seed files to create our tables. And the another to scrape from SQL files. I will put them below for a better description:

```python
import os
import duckdb


class ReadSQLFileError(Exception):
    code='001'


class ReadSeedFilesError(Exception):
    code='002'


def read_sql_file(filepath):
    try:
        with open(filepath, "r") as sql:
            sql_file = sql.read()
        return sql_file
    except ReadSQLFileError as err:
        raise err


def create_tables_from_csv_seeds(connection, seeds_path):
    try:
        for file_name in os.listdir(seeds_path):
            if file_name.endswith(".csv"):
                table_name = os.path.splitext(file_name)[0]
                df = connection.read_csv(os.path.join(seeds_path, file_name))
                connection.register(table_name, df)

    except ReadSeedFilesError as err:
        raise err
```


In [1]:
import duckdb
import utils.__main__ as utils 

I will create a in memory database, i could create a persistent one just changing the database pparameter bellow to a local path.

In [2]:
conn = duckdb.connect(database=":memory:",read_only=False)

Here we create the tables automatically passing the csv seed files.

In [3]:
utils.create_tables_from_csv_seeds(conn,'seeds')

So we have two approaches to solve this for jane using duckdb combined with python. we can do in a procedural way,One is creating a big query sentence, that is not so easy to follow. Other way, is creating variables and storing these SQL snippets. I will do in that way because in the final it generate a simplier visualization:

```sql
SELECT Month, customers.account_id as Account, totalAmountIn, totalAmountOut, (totalAmountIn - totalAmountOut) as Balance
FROM
	({query_in_out_months}) AS totalInAndOut

INNER JOIN
	({query_customers}) as customers
ON
	totalInAndOut.account_id = customers.account_id
    order by Month ASC;

```

In [4]:
query_in_out = utils.read_sql_file('SQL/query_amounts_in_out.sql')
query_time = utils.read_sql_file('SQL/query_time.sql')
query_customers = utils.read_sql_file('SQL/query_customers.sql')
query_in_out_months = utils.read_sql_file('SQL/query_in_out_month.sql').format(query_in_out,query_time)
main_query = utils.read_sql_file('SQL/query.sql').format(query_in_out_months,query_customers)

query_in_out stores the sql from the in_out operations. i used in this part ´case´  sentences to pick values from the in_or_out columns that stores the operations of income and outcome of money in the account. Other thing to note is that i used here UNION ALL, that retrives all info the combined concatenated query.

here is the sql snippet for query_in_out:

```sql
SELECT	
    account_id,
    pix_completed_at as time_id,
    (CASE WHEN in_or_out = 'pix_in' THEN pix_amount END) AS totalAmountIn,
    (CASE WHEN in_or_out = 'pix_out' THEN pix_amount END) AS totalAmountOut
FROM 
    pix_movements
WHERE
    pix_completed_at != 'None'
    
UNION ALL

SELECT 
    account_id,
    transaction_completed_at AS time_id, 
    amount AS totalAmountIn, 
    NULL AS totalAmountOut
FROM 
	transfer_ins
WHERE 
	transaction_completed_at != 'None'
    
UNION ALL
	SELECT 
        account_id,
        transaction_completed_at AS time_id, 
        NULL AS totalAmountIn, 
        amount as totalAmountOut
	FROM
        transfer_outs
	WHERE
         transaction_completed_at != 'None'
```

query_time is the variable that stores the snippet of querying the time sql file. 



```sql
SELECT 
	action_month,
	time_id
FROM 
	(d_time INNER JOIN d_year ON d_time.year_id = d_year.year_id)
INNER JOIN 
	d_month ON d_time.month_id = d_month.month_id
WHERE action_year=2020
```

query_customers variable stores the snippet from query_customers.sql, that retrieves for the customers accounts that we will use in our combined view.

```sql
SELECT
    accounts.account_id 
FROM 
    accounts 
LEFT JOIN 
    customers ON accounts.customer_id  = customers.customer_id
```

i have the basic query, now i will pass one that are a join from  amounts_in_out:

So, finally, Jane has a query that is far better to read. More clean than  the extensive one.

In [5]:
df = conn.sql(main_query)
df

┌───────┬─────────────────────┬────────────────────┬───────────────────┬────────────────────┐
│ Month │       Account       │   totalAmountIn    │  totalAmountOut   │      Balance       │
│ int64 │        int64        │       double       │      double       │       double       │
├───────┼─────────────────────┼────────────────────┼───────────────────┼────────────────────┤
│     1 │ 2922610483805172224 │            1776.69 │               0.0 │            1776.69 │
│     1 │ 2812754350480354304 │            1337.23 │               0.0 │            1337.23 │
│     1 │ 2680031936548642304 │                0.0 │           1933.31 │           -1933.31 │
│     1 │ 2249498282617143808 │                0.0 │            371.82 │            -371.82 │
│     1 │ 1153472375140304896 │            1080.95 │               0.0 │            1080.95 │
│     1 │ 1641327779543874048 │              86.55 │               0.0 │              86.55 │
│     1 │ 1327634959155804928 │            1445.29 │        

### Saving the files

With duckdb to save the file is very simple too just do a ´COPY´ ´TO´ query passing the the name of the file. 

In [6]:
conn.query(f"COPY ({main_query}) TO 'results.csv'")